<a href="https://colab.research.google.com/github/shimul1725/Next-Word-Prediction-Model-using-Python/blob/main/Next_Word_Prediction_Model_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
#Now let’s tokenize the text to create a sequence of words:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
# let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
#Now let’s pad the input sequences to have equal length:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
#Now let’s split the sequences into input and output:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [6]:
#let’s convert the output to one-hot encode vectors:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [7]:
#Now let’s build a neural network architecture to train the model:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
#Now let’s compile and train the model:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 217s 71ms/step - loss: 6.2780 - accuracy: 0.0722
Epoch 2/100
3010/3010 [==============================] - 211s 70ms/step - loss: 5.5690 - accuracy: 0.1176
Epoch 3/100
3010/3010 [==============================] - 214s 71ms/step - loss: 5.1800 - accuracy: 0.1413
Epoch 4/100
3010/3010 [==============================] - 210s 70ms/step - loss: 4.8360 - accuracy: 0.1606
Epoch 5/100
3010/3010 [==============================] - 206s 68ms/step - loss: 4.5158 - accuracy: 0.1794
Epoch 6/100
3010/3010 [==============================] - 207s 69ms/step - loss: 4.2130 - accuracy: 0.2019
Epoch 7/100
3010/3010 [==============================] - 205s 68ms/step - loss: 3.9298 - accuracy: 0.2280
Epoch 8/100
3010/3010 [==============================] - 208s 69ms/step - loss: 3.6607 - accuracy: 0.2603
Epoch 9/100
3010/3010 [==============================] - 203s 68ms/step - loss: 3.4066 - accuracy: 0.2958
Epoch 10/100
3010/3010 [======================

In [9]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 24ms/step
I will leave if they are young mr
